### **BERT sentence embedding**

In [1]:
import torch 
from transformers import BertTokenizerFast, BertModel
import pandas as pd

import logging
import matplotlib.pyplot as plt

2023-03-15 18:38:11.140974: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 18:38:14.069633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /N/soft/rhel8/mvapich/gnu/2.3.5/lib:/N/soft/rhel8/gcc/9.3.0/lib64:/N/soft/rhel8/gcc/9.3.0/lib
2023-03-15 18:38:14.069675: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-15 18:38:34.436442: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic libr

In [2]:
sentences = list('xx')

##### **Gets bert embedding**

In [10]:
def get_sentence_embedding(sentences,
                           save_path=None):
    '''
    Get sentence embeddings
    :param sentences
    '''
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased',
                                      output_hidden_states = True, # Whether the model returns all hidden-states.
                                      )
    model.eval()

    all_data = []
    batch_size = 100
    i = 0
    for idx in range(0, len(sentences), batch_size):
        batch = sentences[idx : min(len(sentences), idx+batch_size)]

        # encoded = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
        encoded = tokenizer.batch_encode_plus(batch,
                                              max_length=50, 
                                              padding='max_length', 
                                              truncation=True)

        # print(encoded)
        encoded = {key:torch.LongTensor(value) for key, value in encoded.items()}
        # print([key for key, value in encoded.items])
        with torch.no_grad():
            outputs = model(**encoded)
        lhs = outputs[2] #all embedding in 13 layer

        #[# layers, # batches, # tokens, # features]
        # token_vecs = lhs[-2]
        # print(len(token_vecs))
        # print(lhs[0].size())

        token_embeddings = torch.stack(lhs, dim=0)
        # print(token_embeddings.size())

        # # token_embeddings = torch.squeeze(token_embeddings, dim=1)

        # # print(token_embeddings.size())
        token_embeddings = token_embeddings.permute(1, 0, 2, 3)
        # print(token_embeddings.size())
        for embedding in token_embeddings:
            #layer token features
            # embedding = embedding.permute(1, 0, 2)
            embedding = embedding[-4:] #

            # print(embedding.size())
            sentence_embedding = torch.sum(embedding, dim=0)


            sentence_embedding = torch.mean(sentence_embedding, dim=0)

            all_data.append(sentence_embedding.tolist())

        if save_path == None:
            return all_data
        
        
        
        return df_emb
    
# get_sentence_embedding(sentences)